In [10]:
!pip install -q anchor-exp
!python -m spacy download en_core_web_lg
!pip install -q transformers
!pip install -q sentence-transformers

from google.colab import drive
drive.mount('/content/drive')

%cd drive/MyDrive/Tesi\ Magistrale\ 204963/Progetto/
 
import warnings
warnings.filterwarnings("ignore")

     |████████████████████████████████| 827.9 MB 1.2 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[Errno 2] No such file or directory: 'drive/MyDrive/Tesi/'
/content/drive/MyDrive/Tesi


In [11]:
import os
import os.path
import numpy as np
import sklearn
import sklearn.model_selection
import sklearn.linear_model
import sklearn.ensemble
import spacy
import sys
from sklearn.feature_extraction.text import CountVectorizer
from anchor import anchor_text
import time

from math import ceil

import torch
import pickle
import nltk
import pandas as pd
import tensorflow as tsf
import matplotlib.pyplot as plt
#from matplotlib_venn_wordcloud import venn2_wordcloud

from autoencoders.model import VAE, DAE, AAE, reparameterize
from autoencoders.vocab import Vocab
from autoencoders.noise import noisy
from autoencoders.utils import set_seed, strip_eos
from autoencoders.batchify import get_batches
from classifier.classifier_nn import build_nn
from nltk.corpus import stopwords
import lime.lime_text
from text_interpretable import *
from coherence import compute_explanations, compare_explanations, compute_score

from sentence_transformers import SentenceTransformer, util
SentenceTransformer('stsb-roberta-large')

nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [12]:
def load_sent(path):
    sents = []
    with open(path) as f:
        for line in f:
            sents.append(line)
    return sents

In [13]:
data = 'yelp'

if data == 'yelp':
  path = 'checkpoints/yelp/daae'
  dataset = 'data/yelp/sentiment'
  class_folder = 'classifier/yelp'

  # Exctract data of interest
  # data_classes = data[(data['stars'] == 1) | (data['stars'] == 5)]
  # x = data_classes['text']
  # y = data_classes['stars'].where(data_classes['stars'] == 1, other=2) - 1
  sents = load_sent(os.path.join(dataset, '1000.pos'))
  num_example = 500 # SOLO 
  sents = sents[:num_example] # SOLO TEST
  n_pos = len(sents)
  sents.extend(load_sent(os.path.join(dataset, '1000.neg')))
  sents = sents[:2*num_example] # SOLO TEST
  y = np.ones(len(sents), dtype=int)
  y[n_pos:] = 0

  vec = pickle.load(open(os.path.join(class_folder, 'vec_nn.pickle'), 'rb'))
  classifier = build_nn(len(vec.vocabulary_))
  classifier.load_weights(os.path.join(class_folder, 'model_dense.hdf5'))
  
  def classifier_fn(s):
    p = classifier.predict(vec.transform(s).toarray())
    return np.append(1-p, p, axis=1)

  cv_d = pickle.load(open('mytest/yelp/new_CV.pickle','rb'))
  score_fabrizio_correzione = pickle.load(open('mytest/yelp/GOW_score.pickle','rb'))

score_dict = dict(zip(cv_d.get_feature_names(), score_fabrizio_correzione))

def score_parola(w):
  if w in score_dict:
    return score_dict[w]
  return 0.0

print("INFO:")
print("\tsents ", type(sents), len(sents))
print("\ty ", type(y), len(y))

INFO:
	sents  <class 'list'> 1000
	y  <class 'numpy.ndarray'> 1000


In [14]:
stop_words = pickle.load(open('stop_words_yelp.pickle', 'rb'))
lt = LemmaTokenizer(split_expression=r'\W+', stop_words=stop_words)
tlt = LimeTextExplainerLatent()
lte = lime.lime_text.LimeTextExplainer()

In [15]:
def score_sim_parola(w, e1, e2):
  # compute similarity scores of two embeddings
  cosine_scores = util.pytorch_cos_sim(e1, e2)
  return abs(cosine_scores.item())

def score_calc(w, e1, e2):
  sim = score_sim_parola(w, e1, e2)
  score = score_parola(w)
  if sim <= 0.3:
    return 2/3*score+sim
  if score <= 0.3:
    return score+2/3*sim
  return score+4/5*sim


def final_score(word, sentence, e1, e2, classifier_fn):
  
  '''
  Compute term relevance scores for a given sentence.
    word: single word or group of words which is a possible good explanation
    sentence: original sentence in lemmatized version 
    e1: embedding of sentence
    e2: embedding of word
    classifier_fn: black-box prediction function
  '''
  sim = abs((util.pytorch_cos_sim(e1, e2)).item()) # vedo quanto è simile il termine nella frase
  score_word = score_parola(word) # se è presente nel cv principale e ha capacità di discernere

  clf_bb =  np.argmax(classifier_fn([sentence]))

  
  present = all(w in sentence for w in word.split()) # se la parola è contenuta nella frase
  similiar = sim >= 0.245
  distinguisher = score_word >= 0.3

  important_absence, important_presence = False, False
  
  if present:
    a = ' '.join([x for x in sentence.split() if x not in word.split()]) # tolgo se presente
    important_absence = np.argmax(classifier_fn([a])) != clf_bb
  else:
    a = sentence + ' ' + word # la inserisco
    a = ' '.join(general_delete_rep(a.split())) # rimuovo doppioni
    important_presence = np.argmax(classifier_fn([a])) != clf_bb

  if present:
    if important_absence:
      return sim+score_word+0.2
    else:
      if similiar:
        return sim+score_word
      else:
        if distinguisher:
          return score_word/1.5
        else:
          return 0.0
  else:
    if important_presence:
      return sim+score_word+0.2
    else:
      if similiar:
        return sim+score_word
      else:
        if distinguisher:
          return score_word/1.5
        else:
          return 0.0

In [16]:
def predict_fn(s):
  p = classifier.predict( vec.transform(s) )
  p = np.append(1-p, p, axis=1)
  return np.argmax( p, axis=1)

In [20]:
nlp = spacy.load('en_core_web_sm')
explainer = anchor_text.AnchorText(nlp, predict_fn, use_unk_distribution=True)
class_names = ['negative', 'positive']

d = {}
for i in range(len(sents)):
  ph = lt([sents[i]])
  #ph = [sents[i]]
  pred = class_names[predict_fn(ph)[0]]
  alternative =  class_names[1 - predict_fn(ph)[0]]
  exp = explainer.explain_instance( ph[0] , predict_fn, threshold=0.99, use_proba=False, batch_size=1)
  anchor = ' '.join(exp.names())
  d[i] = anchor

In [21]:
trf = SentenceTransformer('stsb-roberta-large')
output = []

for i,anchor in d.items():
  if len(anchor) == 0:
    output.append(0.0)
  else:
    ph = lt( [sents[i]] )[0]  
    e1 = trf.encode(ph, convert_to_tensor=True)
    e2 = trf.encode(anchor, convert_to_tensor=True)
    output.append( final_score(anchor,ph,e1,e2,classifier_fn) )

pickle.dump( output, open('../Confronto sLIME/anchor_result.pickle','wb') )